---
# Data Queries
---

Will be using a board game API (https://boardgamegeek.com/xmlapi) to retrieve various information about this week top 50 board games and the reviews left by the users.---


Here is a table of the information retrieved with the API.

| Game information retrieved      | 
|---------------|
| **Username** |
| **Rating** |
| **Comment**|
| **Game Name** | 
| **Mechanics** | 
| **Min Players** | 
| **Max Players** | 
| **Min Playtime** | 
| **Max Playtime** | 
| **Age** | 
| **Average Rating** | 
| **Wanting Count** | 
| **Wishing Count** | 
| **Description** | 
| **Categories** | 

The data will be stored in the csv file [Data](../data/games_comments.csv)

---

### Importing necessary library

In [1]:
import requests
import pandas as pd
import json
from io import StringIO
import os

### Retrieve top 50 games of the week

In [2]:
response = requests.get('https://boardgamegeek.com/xmlapi2/hot?type=boardgame')
response

<Response [200]>

In [3]:
xml_data = StringIO(response.text)

df = pd.read_xml(xml_data)
df

,id,rank,thumbnail,name,yearpublished
0,441696,1,NaN,NaN,NaN
1,244521,2,NaN,NaN,NaN
2,418059,3,NaN,NaN,NaN
3,437356,4,NaN,NaN,NaN
4,372649,5,NaN,NaN,NaN
5,441706,6,NaN,NaN,NaN
6,424785,7,NaN,NaN,NaN
7,349750,8,NaN,NaN,NaN
8,391137,9,NaN,NaN,NaN
9,342942,10,NaN,NaN,NaN


### Retrieving various informations about each top 50 game of the week

In [4]:
# Define filename
file = '../data/games_comments.csv'

# If file exist delete it
if os.path.exists(file):
    os.remove(file)  # Delete the file
    print(f"{file} has been removed.")
else:
    print(f"{file} does not exist.")

# Looping through the top 50 games id of the week
for index, id in enumerate(df['id']) :
    # Define URL for the Queries
    url = f'https://boardgamegeek.com/xmlapi/boardgame/{id}?&comments=1&stats=1'

    # Retrieve Query Response
    response = requests.get(url)

    # Define namespaces 
    namespaces = {'ns': 'http://www.boardgamegeek.com/xmlapi'}

    # Extracting Game Name
    xml_data = StringIO(response.text)
    game_name = pd.read_xml(xml_data, xpath=".//name[@primary='true']", namespaces=namespaces) 
    game_name = game_name['name'].values[0]
  
    # Extracting comments
    xml_data = StringIO(response.text)
    comments = pd.read_xml(xml_data, xpath=".//comment", namespaces=namespaces)

    # Extracting ratings statistics
    xml_data = StringIO(response.text)
    ratings = pd.read_xml(xml_data, xpath=".//ratings", namespaces=namespaces)

    # Extracting board game categories
    xml_data = StringIO(response.text)
    try :
        categories = pd.read_xml(xml_data, xpath=".//boardgame/boardgamecategory", namespaces=namespaces)
        categories = ','.join(categories['boardgamecategory'])
    except:
        categories = 'No category'

    # Extract board game mechanics
    xml_data = StringIO(response.text)
    try :
        mechanics = pd.read_xml(xml_data, xpath=".//boardgamemechanic")
        mechanics = ','.join(mechanics['boardgamemechanic'])
    except:
        mechanics = 'No Mechanics'

    # Retrieving all info under boardgame tag
    xml_data = StringIO(response.text)
    tmp = pd.read_xml(xml_data, xpath=".//boardgame")

    # Retrieving min max player
    min_players = tmp['minplayers'] 
    max_players = tmp['maxplayers'] 

    # Retrieving min max play time
    minplaytime = tmp['minplaytime'] 
    maxplaytime = tmp['maxplaytime'] 

    # Retrieving age
    age = tmp['age'] 

    # Retrieving description
    description = tmp['description']

    # Creating new columns to the comments dataframe to add the inforetrieved about the game
    comments['max_players'] = min_players.values[0]
    comments['max_players'] = max_players.values[0]

    comments['minplaytime'] = minplaytime.values[0]
    comments['maxplaytime'] = maxplaytime.values[0]

    comments['age'] = age[0]

    comments['ratings_avg'] = ratings['average'].values[0]
    
    comments['count_wanting'] = ratings['wanting'].values[0]
    comments['count_wishing'] = ratings['wishing'].values[0]

    comments['description'] = description.values[0]

    comments['categories'] = categories

    comments.insert(3,'gamename', game_name)

    comments.insert(4,'mechanics', mechanics)

    # Add comments dataframe to csv file
    if index == 0 :
        print(0)
        comments.to_csv(file, mode='x', header=True, index=False)
    else :
        comments.to_csv(file, mode='a', header=False, index=False)

../data/games_comments.csv has been removed.
0
